In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

In [3]:
# Download and load the data
import keras
import os

f_path_1 = "data/Euro_Salary.csv"
url_1 = "https://github.com/AkeemSemper/ml_data/raw/main/Euro_Salary.csv"
if not os.path.exists(f_path_1):
    file_1 = keras.utils.get_file(f_path_1, url_1)
df = pd.read_csv(f_path_1)

/Users/ghunaym/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Assignment 2 - Regression
### Predict the TOTAL COMPENSATION for this year. 

The data file provided is a salary survey for tech workers in Europe. We want to predict the total amount of compensation they bring in each year, based off of the details of their work. 

Some notes that will be important:
<ul>
    <li>The total compensation will need to be constructed, there is a column for salary, "Yearly brutto salary (without bonus and stocks) in EUR", as well as a column for bonus compensation, "Yearly bonus + stocks in EUR". 
    <li>Some categorical variables will need some work, and there isn't generally an exact answer. The main concern is things with categories that have a bunch of values with a very small count. For example, if there is only 1 person in City X, then that value likely needs to be addressed. We don't want it encoded into a new column of one 1 and thousands of 0s. 
    <li>There is an article exploring some of the data here: https://www.asdcode.de/2021/01/it-salary-survey-december-2020.html
    <li>Imputation and a bit of data manipulation will be required. 
    <li>Use any regression method you'd like. Some ones are closely related to what we've done, you may want to look at them, e.g. ExtraTreesRegressor. 
    <li>Initial accurracy, and potentially final accuracy, may not be great. When I made a plain model will little optimization the errors were large and the R2 was low. There is lots of room for optimization. 
    <li>Research challenge - try some work on the target, look into TransformedTargetRegressor and see if that helps. Recall in stats when we had skewed distributions... Maybe it helps, maybe it doesn't. 
    <li>EDA and data prep are up to you - you'll probably need to do a little exploring to figure out what cleanup is needed. When I did it, I did things kind of iteratively when I did it. For example, look at the value counts, figure out how to treat the different categories, clean something up, look at the results, potentially repeat if needed. After you figure out what needs to be done, you may be able to take some of those steps and incorporate them into a pipeline to be cleaner....
    <li><b>CRITICAL - Please make sure you publish it after having run it, all the output should be showing.</b>
</ul>

### Details and Deliverables

You'll need to build code to produce the predictions. In particular, there's a few things that'll be marked:
<ul>
    <li>Please add a "presentation version" at the bottom, where you show what you did, and the results. Basically, you start with the original data, you do some work to figure out what's needed, you try a few models and select the best. At the bottom, put what <i>you actually settled on</i>, i.e. after all the figuring and exploring, here's the code that goes from raw data to final results, and here's what the results were. I should be able to read this part and understand what you did clearly:
    <ul>
        <li> Please make a pipeline that does the prep work - you may need some exploration or several trials before settling on what exactly to use, that's normal. Once you've settled, build that into a pipeline so it's clear and repeatable.
        <li> What you settled on for data cleaning, along with what prompted it. 
        <li> Feature Selection - Please identify what you did for feature selection. No need for a long explaination, something along the lines of "I did X, and the result was that 4 features were removed". Try at least 2 things. 
        <li> Model selection - between selecting a model style and tuning it with hyperparameters, what did you test and what won?
        <li> Overall, how good was your model and what things may make sense to try to do even better? 
        <li> If you could use titles/bullet points I'd really appreciate it. 
    </ul>
    <li>Grade Breakdown:
    <ul>
        <li> Code is readable, there are comments: 20%
        <li> Explaination as defined above: 60% (20% each point)
        <li> Accuracy: 20% As compared to everyone else. This will be generously graded, I won't be surprised if overall accuracy is low for most people. 
    </ul>
</ul>

<b>The biggest challenge here is translating the data into something useful and clean. This will probably require a bit of exploration, examining the data, thinking about what it means, trying something, then making a model to see what the results are. In particular, think about what value some of the less clean bits of data my hold - binning/grouping, numerical transformations, outlier removal, etc... are all likely to be useful somewhere. You almost certainly need to look at it column by column and make a decision, I'll apologize up front, it isn't the most fun process in the world. There is not one specific correct answer.</b>

In [4]:
#Load Data
#df = pd.read_csv("data/Euro_Salary.csv")
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Timestamp,1253,1248,24/11/2020 13:55:19,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,1226.0,NaN,NaN,NaN,32.509788,5.663804,20.0,29.0,32.0,35.0,69.0
Gender,1243,3,Male,1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,1253,119,Berlin,681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Position,1247,148,Software Engineer,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total years of experience,1237,48,10,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Years of experience in Germany,1221,53,2,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Seniority level,1241,24,Senior,565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Your main technology / programming language,1126,256,Java,184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other technologies/programming languages you use often,1096,562,Javascript / Typescript,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df[393:396]

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
393,24/11/2020 19:15:02,30.0,Male,Moscow,Software Engineer,5,0,Middle,C,C/C++,14712.0,0,30,Full-time employee,Unlimited contract,Russian,101-1000,Product
394,24/11/2020 19:15:49,33.0,Male,Berlin,Product Manager,5,5,Senior,NaN,Python,70000.0,800,30,Full-time employee,NaN,German,101-1000,Product
395,24/11/2020 19:19:30,35.0,Male,Berlin,QA Engineer,11,10,Senior,Java,"Python, Javascript / Typescript, .NET, Java / ...",74400.0,NaN,30,Full-time employee,Unlimited contract,English,101-1000,Product


In [6]:
#df.info()
df["Other technologies/programming languages you use often"].value_counts()

Other technologies/programming languages you use often
Javascript / Typescript                                                                   44
Python                                                                                    37
SQL                                                                                       31
AWS, Docker                                                                               16
Kotlin                                                                                    15
                                                                                          ..
Python, Ruby, Java / Scala, Go, Rust, AWS, Kubernetes, Docker, Terraform, helm, packer     1
Kotlin, SQL, AWS, Kubernetes, Docker                                                       1
Python, R, Go                                                                              1
C/C++, PHP, SQL                                                                            1
Python, AWS, Ku

In [7]:
df.sample(20)

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
764,26/11/2020 11:50:35,31.0,Male,Berlin,ML Engineer,7,2,Middle,Python,"AWS, Kubernetes, Docker",80000.0,12000,28,Full-time employee,Unlimited contract,English,1000+,Product
804,26/11/2020 22:41:33,20.0,Male,Berlin,Mobile Developer,5,2,Lead,C#,.NET,240000.0,15000+-,27,Full-time employee,Unlimited contract,English,1000+,Product
454,24/11/2020 22:22:57,33.0,Male,Berlin,Software Engineer,NaN,10,Senior,Java,"Java / Scala, Google Cloud, Kubernetes, Docker",90000.0,9000,25,Full-time employee,Unlimited contract,English,1000+,Product
1146,11/12/2020 16:25:59,28.0,Male,Berlin,ML Engineer,5,3,Middle,"Python, Pytorch","Python, AWS, Google Cloud, Docker",54000.0,10000,27,Full-time employee,Unlimited contract,English,up to 10,Startup
1048,02/12/2020 00:27:03,31.0,Male,Berlin,Software Engineer,7,2,Senior,Java,"SQL, AWS, Kubernetes, Docker",75000.0,12500,30,Full-time employee,Unlimited contract,English,101-1000,Product
793,26/11/2020 21:13:28,32.0,Male,Berlin,Senior Data Engineer,9,4,Senior,Python,"Java / Scala, SQL, AWS",90000.0,NaN,30,Full-time employee,Unlimited contract,English,51-100,Startup
1222,05/01/2021 22:56:24,34.0,Male,Ingolstadt,Software Engineer,3.5,3.5,Junior,C++,"Python, C/C++, Docker",55000.0,55000,30,Full-time employee,Unlimited contract,German,1000+,Consulting / Agency
150,24/11/2020 12:12:35,25.0,Male,Berlin,Backend Developer,4,1,Middle,Python,"SQL, Go",60000.0,NaN,24,Full-time employee,Unlimited contract,English,11-50,Startup
1067,03/12/2020 08:53:41,32.0,Male,Berlin,Software Engineer,13,5,Lead,JavaScript,"Javascript / Typescript, SQL, AWS, Docker",83000.0,3000,30,Full-time employee,Unlimited contract,English,1000+,Product
220,24/11/2020 13:32:34,36.0,Male,Berlin,Engineering Manager,7,1,Lead,"Swift, objective-c","Kotlin, Swift, Go, AWS, Kubernetes, Docker",80000.0,95000,26,Full-time employee,Unlimited contract,English,1000+,Product


In [17]:
import numpy as np
import pandas as pd

base_salary = "Yearly brutto salary (without bonus and stocks) in EUR"
bonus_col = "Yearly bonus + stocks in EUR"

# Convert to numeric safely
df[base_salary] = pd.to_numeric(df[base_salary], errors="coerce")
df[bonus_col] = pd.to_numeric(df[bonus_col], errors="coerce")

# Fill bonus NaNs with 0
df[bonus_col] = df[bonus_col].fillna(0)

# Drop rows missing base salary
df = df.dropna(subset=[base_salary])

# Target
df["TOTAL_COMP"] = df[base_salary] + df[bonus_col]

df[["TOTAL_COMP", base_salary, bonus_col]].describe()

,TOTAL_COMP,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR
count,1.253000e+03,1.253000e+03,1.253000e+03
mean,8.428361e+07,8.027904e+07,4.004569e+06
std,2.829182e+09,2.825061e+09,1.412516e+08
min,1.000100e+04,1.000100e+04,0.000000e+00
25%,6.000000e+04,5.880000e+04,0.000000e+00
50%,7.400000e+04,7.000000e+04,0.000000e+00
75%,9.100000e+04,8.000000e+04,9.400000e+03
max,1.000000e+11,1.000000e+11,5.000000e+09


In [28]:
# Drop rows missing base salary
df = df.dropna(subset=[base_salary])

# Target
df["TOTAL_COMP"] = df[base_salary] + df[bonus_col]

df[["TOTAL_COMP", base_salary, bonus_col]].describe()

# Drop OUtleirs
low, high = df["TOTAL_COMP"].quantile([0.01, 0.99])

df = df[(df["TOTAL_COMP"] >= low) & (df["TOTAL_COMP"] <= high)].copy()

print("Kept range:", low, "to", high)
df["TOTAL_COMP"].describe()

Kept range: 28081.0 to 218650.0000000001


count      1202.000000
mean      81642.002978
std       32244.427740
min       28300.000000
25%       62000.000000
50%       74000.000000
75%       90000.000000
max      215000.000000
Name: TOTAL_COMP, dtype: float64

In [ ]:
def group_rare_categories(df, col, threshold=100, other_label="Other"):
    vc = df[col].astype(str).fillna("Missing").value_counts()
    rare_values = vc[vc < threshold].index
    df[col] = df[col].astype(str).fillna("Missing").replace(rare_values, other_label)
    return df

candidate_cols = ["City", "City ", "Position", "Position ", "Country", "Country of residence", "Company size"]

# Keep only the ones that exist in this dataset
high_card_cols = [c for c in candidate_cols if c in df.columns]

print("High-card cols found:", high_card_cols)

# Group rare categories
for c in high_card_cols:
    df = group_rare_categories(df, c, threshold=100)

# preview
df[high_card_cols].head()

High-card cols found: ['City', 'Position ', 'Company size']


,City,Position,Company size
0,Munich,Software Engineer,51-100
1,Berlin,Backend Developer,101-1000
3,Berlin,Other,51-100
4,Berlin,Backend Developer,101-1000
5,Berlin,Other,11-50


In [ ]:
# Columns that might cause feature explosion
cols_to_drop = [
    "Timestamp",        
    "City",             
    "Position "           
]

X = df.drop(columns=[base_salary, bonus_col, "TOTAL_COMP"] + cols_to_drop, errors="ignore")
y = df["TOTAL_COMP"]

X.shape, y.shape

((1202, 13), (1202,))

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Identify types
numeric_features = X.select_dtypes(include=["int64", "float64"]).columns
categorical_features = X.select_dtypes(include=["object"]).columns

numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", min_frequency=10))
])

preprocessor = ColumnTransformer([
    ("num", numeric_pipeline, numeric_features),
    ("cat", categorical_pipeline, categorical_features)
])

In [32]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_model = Pipeline([
    ("preprocess", preprocessor),
    ("model", LinearRegression())
])

lin_model.fit(X_train, y_train)
lin_preds = lin_model.predict(X_test)

lin_rmse = np.sqrt(mean_squared_error(y_test, lin_preds))
print("Linear Regression RMSE:", lin_rmse)

Linear Regression RMSE: 33936.75005504474


In [34]:
from sklearn.linear_model import Lasso

lasso_model = Pipeline([
    ("preprocess", preprocessor),
    ("model", Lasso(alpha=0.1, max_iter=50000, random_state=42))
])

lasso_model.fit(X_train, y_train)
lasso_preds = lasso_model.predict(X_test)

lasso_rmse = np.sqrt(mean_squared_error(y_test, lasso_preds))
print("Lasso RMSE:", lasso_rmse)

Lasso RMSE: 34033.23299170564


In [35]:
alphas = [0.01, 0.05, 0.1, 0.2, 0.5, 1.0]

for a in alphas:
    model = Pipeline([
        ("preprocess", preprocessor),
        ("model", Lasso(alpha=a, max_iter=50000, random_state=42))
    ])
    scores = cross_val_score(
        model, X_train, y_train,
        scoring="neg_root_mean_squared_error",
        cv=5
    )
    print(f"alpha={a:<4}  CV RMSE={-scores.mean():,.0f}")

/Users/ghunaym/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:656: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 21708027503.91217, tolerance: 72507784.46578053
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/ghunaym/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:656: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4302076491.571106, tolerance: 76734107.99772382
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/ghunaym/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:656: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3401831910.5740356, tolerance: 74243671.7183829
  model = cd_fast.sparse_enet_coordinate_descent(


alpha=0.01  CV RMSE=27,585
alpha=0.05  CV RMSE=27,567
alpha=0.1   CV RMSE=27,567
alpha=0.2   CV RMSE=27,523
alpha=0.5   CV RMSE=27,521
alpha=1.0   CV RMSE=27,518


# Answers and Explainations
(Expand/modify as needed)

### Here's the Data Cleaning Steps I Used

- First, I converted the salary and bonus columns to numeric values because some entries were stored as text. I filled missing bonus values with 0 and removed rows where base salary was missing since it is required to calculate total compensation.
- I created a new column called TOTAL_COMP by adding base salary and bonus together.
- When looking at the summary statistics, I noticed some extremely large salary values that were not realistic. To prevent these outliers from affecting the model too much, I removed observations outside the 1st and 99th percentile range.
- For categorical columns like City and Position that had many unique values, I grouped rare categories into an “Other” category to reduce noise and keep the model more stable.

### Here's my Tuning/Feature Selection Steps

- I used Lasso regression to help reduce overfitting and handle the large number of categorical features created from one-hot encoding.
- I tested different alpha values (0.01 to 1.0) using 5-fold cross-validation and compared their RMSE scores. As alpha increased, the model became more regularized. The best cross-validation result occurred at alpha = 1.0.
- Grouping rare categories and removing outliers also helped reduce convergence warnings and improve model stability.

### Here's my Model's Performance

- The Linear Regression model achieved a test RMSE of about 33,937.
- The Lasso model achieved a test RMSE of about 34,033.
- Using cross-validation, the best alpha value (1.0) produced a CV RMSE of approximately 27,518.
- This means that, on average, the model’s predictions are within about 27k–34k of the true salary values.

### Here's my Final Conclusion on What Worked Best
- The biggest improvements came from cleaning the data properly. Removing extreme outliers and grouping rare categories had a noticeable impact on model performance.
- Lasso and Linear Regression performed very similarly, but Lasso with alpha = 1.0 gave slightly better cross-validation results and helped control noisy features.
- Overall, careful preprocessing made more difference than changing the model itself.